In [62]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import PIL

In [3]:
conv_base = ResNet50(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

c:\users\asadullah noor\appdata\local\programs\python\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 364s 4us/step


In [5]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.3,
height_shift_range=0.3,
shear_range=0.3,
zoom_range=0.3,
horizontal_flip=True,
fill_mode='nearest')

In [12]:
from keras import models
from keras import layers

In [13]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [15]:
from PIL import Image
from PIL import Image as pil_image
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
'images',
target_size=(150, 150),
batch_size=20,
class_mode='binary')
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=2e-5),
metrics=['acc'])
history = model.fit_generator(
train_generator,
steps_per_epoch=10,
epochs=10,)

Found 5 images belonging to 2 classes.
Epoch 1/10
10/10 [==============================] - 12s 1s/step - loss: 0.2689 - acc: 0.8800
Epoch 2/10
10/10 [==============================] - 9s 899ms/step - loss: 0.0463 - acc: 0.9800
Epoch 3/10
10/10 [==============================] - 9s 917ms/step - loss: 0.0101 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 9s 927ms/step - loss: 0.0406 - acc: 0.9600
Epoch 5/10
10/10 [==============================] - 9s 926ms/step - loss: 0.0022 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 10s 951ms/step - loss: 0.0172 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 10s 964ms/step - loss: 7.9884e-04 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 9s 941ms/step - loss: 0.0014 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 9s 924ms/step - loss: 2.0717e-04 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 9s 933ms/step - loss: 0.0022 - acc: 1.0000


In [16]:

import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.title('Training accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training loss')
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [63]:
from PIL import Image
import numpy as np
path = 'test/cat.jpg'
def load_image(path):
    img = Image.open(path)
    img.load()
    img = img.resize((150, 150), PIL.Image.ANTIALIAS)

    data = np.asarray( img, dtype="int32" )
    print(data.shape)
    return data.reshape(1,150,150,3)

In [65]:
from PIL import Image

In [66]:
model.predict(load_image('test/cat.jpg'))

(150, 150, 3)


array([[0.02865042]], dtype=float32)

In [67]:
model.predict(load_image('test/dog.jpg'))

(150, 150, 3)


array([[0.11081013]], dtype=float32)